In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.integrate import odeint

In [3]:
t, g, m, l = sp.symbols(r't g m l')

In [4]:
theta , phi = sp.Function(r'\theta')(t), sp.Function(r'\phi')(t)

In [5]:
theta_dot = sp.diff(theta,t)
phi_dot = sp.diff(phi,t)

theta_ddot = sp.diff(theta_dot,t)
phi_ddot = sp.diff(phi_dot,t)

In [6]:
x = l*sp.sin(theta)*sp.cos(phi)
y = l*sp.sin(theta)*sp.sin(phi)
z = l*sp.cos(theta)

In [7]:
x_f = sp.lambdify((theta,phi,l),x)
y_f = sp.lambdify((theta,phi,l),y)
z_f = sp.lambdify((theta,phi,l),z)

In [8]:
v_x = sp.diff(x,t)
v_y = sp.diff(y,t)
v_z = sp.diff(z,t)

In [9]:
# Kinetic Energy
T = (1/2)*m*(v_x**2 + v_y**2 + v_z**2)
# Potential Energy
V = -m*g*z
# Lagrangian
L = T - V

In [10]:
LE1 = sp.diff(sp.diff(L,theta_dot),t) - sp.diff(L,theta)
LE2 = sp.diff(sp.diff(L,phi_dot),t) - sp.diff(L,theta)

In [11]:
# p_theta = sp.diff(L,theta_dot)
# p_dot_theta = sp.diff(L,theta)

# p_phi = sp.diff(L,phi_dot)
# p_dot_phi = sp.diff(L,phi)

In [12]:
# sol = sp.solve([p_theta, p_dot_theta, p_phi, p_dot_phi],(theta_dot,theta_ddot, phi_dot, phi_ddot))

In [13]:
sol = sp.solve([LE1,LE2],(theta_ddot,phi_ddot))

We have two 2$^{nd}$ order differential equations. We need to reduce it into four 1$^{st}$ order differential equations
$$\frac{d^2\theta}{dt^2} = \cdots $$
$$\frac{d^2\phi}{dt^2} = \cdots$$
$\psi $is the velocity along $\theta$
$\xi $ is the velocity along $\phi$
$\frac{d\theta}{dt} = \psi$
$\frac{d\psi}{dt} = \frac{d^2\theta}{dt^2} $
$\frac{d\phi}{dt} = \xi$
$\frac{d\xi}{dt}=\frac{d^2\phi}{dt^2} $

In [14]:
dpsidt_f = sp.lambdify((t,g,m,l,theta,phi,theta_dot,phi_dot),sol[theta_ddot])
dxidt_f = sp.lambdify((t,g,m,l,theta,phi,theta_dot,phi_dot),sol[phi_ddot])
dthetadt_f = sp.lambdify((theta_dot),theta_dot)
dphidt_f = sp.lambdify(phi_dot,phi_dot)

In [15]:
def dSdt(S,t,g,m,l):
    theta, phi, psi, xi = S
    return [
        dthetadt_f(psi),
        dphidt_f(xi),
        dpsidt_f(t,g,m,l,theta,phi,psi,xi),
        dxidt_f(t,g,m,l,theta,phi,psi,xi)
    ]

In [16]:
initial = [np.pi/6,np.pi/5,3.1,3]
t = np.linspace(0,60,1000)
m = 2
l = 5
g = 9.8
ans = odeint(dSdt,y0=initial,t=t,args=(g,m,l))

In [17]:
ans.shape

(1000, 4)

In [18]:
theta_val = ans.T[0]
phi_val = ans.T[1]
psi_val = ans.T[2]
xi_val = ans.T[3]

In [19]:
def get_pos(theta,phi,l):
    return [x_f(theta,phi,l), y_f(theta,phi,l), z_f(theta,phi,l)]

x_val , y_val , z_val = get_pos(theta_val,phi_val,l)

In [20]:
np.save('D:/dataset/circular_pendulum.npy',[x_val,y_val,z_val])